# CSMCC16 Flight Coursework

### Import packages

In [47]:
import numpy as np
import pandas as pd
import datetime
import re
from difflib import SequenceMatcher

### Import data

In [48]:
#import csv as dataframe
pass_data = pd.DataFrame(pd.read_csv('AComp_Passenger_data.csv',header=None))
airport_lat_lon = pd.DataFrame(pd.read_csv('Top30_airports_LatLong.csv',header=None))

In [49]:
# Add Column Names
pass_data.columns = ['Passenger_id'
                        ,'Flight_id'
                        ,'From_airport_IATA'
                        ,'Dest_airport_IATA'
                        ,'Depart_time_epoch'
                        ,'Flight_time_min']
airport_lat_lon.columns = ['Airport_name'
                        ,'Airport_IATA_code'
                        ,'Latitude'
                        ,'Longitude']

### Remove missing values

In [50]:
#Remove values where passenger id and flight id are missing
pass_data=pass_data.dropna(subset=['Flight_id','Passenger_id'])

### Functions for Additional Columns and formats

In [51]:
#calculate datetime values from epoch and arrival time
def calc_arrive_time(df):
    df['Depart_time_datetime'] = pd.to_datetime(df['Depart_time_epoch'],unit='s')
    df['Arrive_epoch'] = pass_data['Depart_time_epoch']+df['Flight_time_min']*60
    df['Arrive_time_datetime'] = pd.to_datetime(df['Arrive_epoch'],unit='s')

In [63]:
#create route (to use as unique id)
def route(df):
    df['Route'] = df['From_airport_IATA']+'-' + df['Dest_airport_IATA']+'-' + df['Depart_time']

In [73]:
#break route for details
def break_route(df):
    df[['From_airport_IATA','Dest_airport_IATA','Depart_time']]=df['Route'].str.split('-',expand=True)

In [74]:
#break_route(pass_data_route)
#pass_data_route[['From_airport_IATA','Dest_airport_IATA','Depart_time']]=pass_data_route['Route'].str.split('-',expand=True)

In [11]:
#Add new datetime formats
def add_times(df):
    df['Depart_day_month']=df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y")
    #df['Depart_day_datetime']=pass_data_df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y %H:%M:%S")
    df['Depart_time']=df['Depart_time_datetime'].dt.strftime("%H:%M:%S")
    df['Arrive_time']=df['Arrive_time_datetime'].dt.strftime("%H:%M:%S")
    df['Depart_day_month']=df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y")
    #pass_data_df['Depart_day_datetime']=pass_data_df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y %H:%M:%S")
    df['Depart_time']=df['Depart_time_datetime'].dt.strftime("%H:%M:%S")
    df['Arrive_time']=df['Arrive_time_datetime'].dt.strftime("%H:%M:%S")

In [14]:
#Add column to allow counting of passengers
def add_1(df):
    df['count']='1'

### Add Additional Columns and formats

In [53]:
calc_arrive_time(pass_data)

In [12]:
add_times(pass_data)

In [45]:
# route(flight_only_data)

In [15]:
####check if necessary!!
add_1(pass_data)

In [16]:
# pass_data

In [ ]:
route(pass_data)

In [17]:
# Select Route info only, and group data
key_route_info=['Route'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_day_month'
                      ,'Depart_time'
                      ,'Flight_time_min'
                      ,'Arrive_time']
route_info=pass_data[key_route_info]
route_info=route_info.groupby(key_route_info).count()
route_info.reset_index(inplace=True)

In [56]:
# def group(base_df,variables,new_df):
#     new_df=pd.DataFrame()
#     #select variables only
#     new_df=base_df[variables]
#     #group by variables to create a table
#     new_df=new_df.groupby(variables).count()
#     #reset index
#     new_df.reset_index(inplace=True)
    
# key_route_info=['Route'
#                       ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_day_month'
#                       ,'Depart_time'
#                       ,'Flight_time_min'
#                       ,'Arrive_time']
# route_info_new=pd.DataFrame()
# group(pass_data,key_route_info,route_info_new)

In [18]:
# route_info

In [19]:
# Number of passengers per flight id
flight_pass = ['Flight_id','Passenger_id']
flight_pass_data = pass_data[flight_pass]
pass_per_flight=flight_pass_data.groupby('Flight_id').count()
pass_per_flight.reset_index(inplace=True)
#pass_per_flight

### Flight data only

In [20]:
#select subset of data 
flight_info = ['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time'         
                        ]
flight_only_data = pass_data[flight_info]
#Group data by flight
flight_only_data = flight_only_data.groupby(flight_info).count()
#reset index
flight_only_data.reset_index(inplace=True)

## Error Correction
### Flights

In [21]:
#check data
# flight_only_data

In [22]:
#correct formats
format_IATA = r"[A-Z][A-Z][A-Z]"
format_flight_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z]"

#format_pass_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z][A-Z][0-9]"
#flight_only_data['Pass_id_format'] =flight_only_data['Passenger_id'].str.contains(format_pass_id)

#add column indicating if correct
flight_only_data['From_airport_format'] =flight_only_data['From_airport_IATA'].str.contains(format_IATA)
flight_only_data['Dest_airport_format'] =flight_only_data['Dest_airport_IATA'].str.contains(format_IATA)
flight_only_data['Flight_id_format'] =flight_only_data['Flight_id'].str.contains(format_flight_id)

In [23]:
#add column indicating if all flight regex correct
flight_only_data['Flight_info_correct']=flight_only_data.all(axis='columns')

In [24]:
#select only those whose regex is correct
flight_info_true = flight_only_data.loc[flight_only_data['Flight_info_correct'] == True]

In [25]:
#Take True airport values, change column names to enable merge

true_airports_from = pd.DataFrame(airport_lat_lon.Airport_IATA_code.unique())
#rename columns to allow merge
true_airports_from.columns = ['From_airport_IATA']
#add column to know if exists
true_airports_from['From_airport_exists']=True

true_airports_dest = pd.DataFrame(airport_lat_lon.Airport_IATA_code.unique())
#rename columns to allow merge
true_airports_dest.columns = ['Dest_airport_IATA']
#add column to know if exists
true_airports_dest['Dest_airport_exists']=True

In [26]:
#merge with from
merge_with_from=flight_info_true.merge(true_airports_from
                     , how='outer', on=['From_airport_IATA'])
#merge with dest
merge_with_dest=merge_with_from.merge(true_airports_dest
                     , how='outer', on=['Dest_airport_IATA'])
#check data
#merge_with_dest

In [27]:
#drop NaNs - any that are not in the true airport list
merge_with_dest=merge_with_dest.dropna(subset=['From_airport_exists', 'Dest_airport_exists'])

In [28]:
# merge_with_dest

In [29]:
#select only flight info 
flight_only_data = merge_with_dest[flight_info]
#add a column to allow count
#add_1(flight_only_data)
#Group data by flight
flight_only_data = flight_only_data.groupby(flight_info).sum()
#reset index
flight_only_data.reset_index(inplace=True)

In [31]:
#merge correct flights with passenger numbers
flight_only_data = flight_only_data.merge(pass_per_flight, how='inner', on=['Flight_id'])

In [32]:
# flight_only_data

In [33]:
#add route info to flight only data
route(flight_only_data)

In [34]:
#sort, route in alpha-numeric order, count of passenger id in descending (lowest value last)
flight_only_data=flight_only_data.sort_values(by=['Route','Passenger_id']
                             , ascending=[True,False]
                             , na_position='first')

In [35]:
# Remove duplicate with lowest number of passenger ids
flight_only_data['Route_dupe']=flight_only_data['Route'].duplicated()
flight_only_deduped = flight_only_data[flight_only_data.Route_dupe == False]

In [36]:
len(flight_only_deduped)

30

## add to passenger data

In [37]:
# #Add route info to pass_data
# route(pass_data)

In [38]:
#Select data required
pass_data_slim = pass_data[['Passenger_id','Flight_id','Route']]
flight_id_route = flight_only_deduped[['Route','Flight_id']]

In [39]:
#Merge correct flight id
flight_id_route.columns = ['Route','correct_Flight_id']
pass_data_route = pass_data_slim.merge(flight_id_route, how='outer', on=['Route'])
pass_data_route.loc[pass_data_route['correct_Flight_id'].notna(), 'Final_Flight_id'] = pass_data_route['correct_Flight_id']
pass_data_route.loc[pass_data_route['correct_Flight_id'].isnull(), 'Final_Flight_id'] = pass_data_route['Flight_id']

In [40]:
#Merge correct route info
flight_id_route.columns = ['Correct_Route','Flight_id']
pass_data_route = pass_data_route.merge(flight_id_route, how='outer', on=['Flight_id'])
pass_data_route.loc[pass_data_route['Correct_Route'].notna(), 'Final_Route'] = pass_data_route['Correct_Route']
pass_data_route.loc[pass_data_route['Correct_Route'].isnull(), 'Final_Route'] = pass_data_route['Route']

In [41]:
#Select only correct data and rename
pass_data_route = pass_data_route[['Passenger_id','Final_Flight_id','Final_Route']]
pass_data_route.columns = ['Passenger_id','Flight_id','Route']

## Error Correction
### Passenger

In [ ]:
format_pass_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z][A-Z][0-9]"
pass_data_route_test=pass_data_route
pass_data_route_test['Pass_id_format'] =pass_data_route['Passenger_id'].str.contains(format_pass_id)
pass_data_route_test

In [ ]:
#add column to count
pass_data_route_test['count'] = '1'

In [ ]:
#select subset of data 
pass_data_route_test = pass_data_route_test[['Passenger_id'
                   #   ,'Flight_id'
                      ,'Pass_id_format'
                      ,'count'
                        ]]

In [ ]:
#Group data to see passengers and total flights by passenger id
pass_only=pass_data_route_test.groupby(['Passenger_id'
                        ,'Pass_id_format'
                         ]).count()
#reset index
pass_only.reset_index(inplace=True)

pass_only

In [ ]:
#select only those whose regex is correct 
## this is not necessary as the second code will remove these, 
## however it is left here as this will be a better way to check if correct passenger id
pass_info_true = pass_only.loc[pass_only['Pass_id_format'] == True]

In [ ]:
#select only those who have more than 1 flight
pass_info_true = pass_only.loc[pass_only['count'] != 1]
#pass_info_true

In [ ]:
pass_info_false = pass_only.loc[pass_only['count'] == 1]
#pass_info_false

In [ ]:
#Create list of false passenger ids
false = pass_info_false['Passenger_id']
false

In [ ]:
#Create list of false passenger ids
true = pass_info_true['Passenger_id']
#true

In [ ]:
#create function to find the best match for a given false value and list for true values
def best_match(f,t):
    global test_best_true
    global final_true
    test_best_true = []
    final_true =[]
    for t in true:
        test_best_true.append([t,round(SequenceMatcher(None, t, f).ratio(),3)])
    test_best_true = pd.DataFrame(test_best_true)
    max_true = test_best_true.iloc[test_best_true[1].idxmax(), 0]
    final_true.append([f,max_true])
    #final_true = pd.DataFrame (final_true)
   # print(final_true)

In [ ]:
#test each false value
best_match_results = []
for f in false:
    best_match(f,t)
    best_match_results.append(final_true)

In [ ]:
#turn into dataframe
best_match_results_pd=pd.DataFrame (best_match_results)
best_match_results_pd.columns = ['results']
best_match_results_pd
#obj_1_list.columns = ['From_airport_IATA']

In [ ]:
#pass_data_route['From_airport_IATA']
#best_match_results_pd['results'].str.split('-',expand=True)
#route(pass_data_route)
pass_data_route
route_info

## Objective 1: 
### Determine the number of flights from each airport; include a list of any airports not used.

In [81]:
airport_list = airport_lat_lon['Airport_IATA_code']
airport_list
break_route(pass_data_route)
from_list = pass_data_route['From_airport_IATA']

In [214]:
#mapper, where key = airport and value = 1 for each airport in from_list
key2val2 = [[a, 1] for a in from_list]
mapper = pd.DataFrame(key2val2, columns=['Airport_IATA_code','value'])
#mapper

#reducer - takes key and val from mapper to provide final key and value
reducer=mapper.groupby(['Airport_IATA_code']).sum()
reducer
reducer.reset_index(inplace=True)

In [216]:
# merge full list of airports with number of flights from each airport
airport_list=pd.DataFrame(airport_list)
num_flights = airport_list.merge(reducer, how='outer', on=['Airport_IATA_code'])

In [218]:
# clean up results
num_flights=num_flights.fillna(0)
num_flights=num_flights.sort_values(by=['value','Airport_IATA_code'], ascending=[False,True])
num_flights.set_index('Airport_IATA_code', inplace=True, drop=True)
num_flights

,value
Airport_IATA_code,
DEN,46.0
CAN,37.0
IAH,37.0
ATL,36.0
KUL,33.0
ORD,33.0
CGK,27.0
JFK,25.0
LHR,25.0


## Objective 2
### Create a list of flights based on the Flight id, this output should include the passenger Id, relevant IATA/FAA codes, the departure time, the arrival time (times to be converted to HH:MM:SS format), and the flight times.

In [ ]:
#pass_data_route
calc_arrive_time(pass_data_route)
#num_flights = airport_list.merge(from_counts_df, how='outer', on=['Airport_IATA_code'])

In [221]:
flights = pass_data_route['Flight_id']

In [240]:
#mapper, where key = airport and value = 1 for each airport in from_list
key2val2 = [[a, 1] for a in flights]
mapper = pd.DataFrame(key2val2, columns=['Flight_id','value'])
#mapper

#reducer - takes key and val from mapper to provide final key and value
num_passengers=mapper.groupby(['Flight_id']).sum()
#num_passengers
#reducer
num_passengers.reset_index(inplace=True)

In [239]:
# clean up results
#num_passengers=num_flights.fillna(0)
num_passengers=num_passengers.sort_values(by=['value','Flight_id'], ascending=[False,True])
num_passengers.set_index('Flight_id', inplace=True, drop=True)
num_passengers

,value
Flight_id,
ULZ8130D,27
GMO5938W,25
XXQ4064B,25
KJR6646J,23
QHU1140O,21
SQU6245R,21
WSK1289Z,21
FYL5866L,20
JVY9791G,20


## Objective 3
### Calculate the number of passengers on each flight.

In [ ]:
#obj2_results = 
pass_data_route.groupby(['Route'
                      ,'Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_day_month'
                      ,'Depart_time'
                      ,'Arrive_time'
                      ,'Flight_time_min']).count()
#reset index as columns
#obj2_results.reset_index(inplace=True)
#obj2_results

## Objective 4
### Calculate the line-of-sight (nautical) miles for each flight and the total travelled by each passenger and thus output the passenger having earned the highest air miles.

In [242]:
#flight info
flight_from = flight_only_deduped[['Flight_id','From_airport_IATA']]
flight_dest = flight_only_deduped[['Flight_id','Dest_airport_IATA']]

In [243]:
#rename columns to allow merge
flight_from.columns = ['Flight_id'
                        ,'Airport_IATA_code']
flight_dest.columns = ['Flight_id'
                        ,'Airport_IATA_code']

In [244]:
#merge from flight with latitude and longitude
from_lat_long = flight_from.merge(airport_lat_lon, how='inner', on=['Airport_IATA_code','Airport_IATA_code'])
from_lat_long.columns = ['Flight_id'
                        ,'From_Airport_IATA_code'
                        ,'From_Airport_Name'
                        ,'From_Lat'
                        ,'From_Lon'
                    ]

In [245]:
#merge dest flight with latitude and longitude
dest_lat_lon = flight_dest.merge(airport_lat_lon, how='inner', on=['Airport_IATA_code','Airport_IATA_code'])
dest_lat_lon.columns = ['Flight_id'
                        ,'Dest_Airport_IATA_code'
                        ,'Dest_Airport_Name'
                        ,'Dest_Lat'
                        ,'Dest_Lon'
                    ]

In [246]:
#merge from flight with latitude and longitude
from_dest_lat_lon = from_lat_long.merge(dest_lat_lon, how='outer', on=['Flight_id','Flight_id'])

In [247]:
from_dest_lat_lon

,Flight_id,From_Airport_IATA_code,From_Airport_Name,From_Lat,From_Lon,Dest_Airport_IATA_code,Dest_Airport_Name,Dest_Lat,Dest_Lon
0,ATT7791R,AMS,AMSTERDAM,52.308613,4.763889,DEN,DENVER,39.861656,-104.673178
1,FYL5866L,ATL,ATLANTA,33.636719,-84.428067,HKG,HONG KONG,22.308919,113.914603
2,XOY7948U,ATL,ATLANTA,33.636719,-84.428067,LHR,LONDON,51.477500,-0.461389
3,YZO4444S,BKK,BANGKOK,13.681108,100.747283,MIA,MIAMI,25.793250,-80.290556
4,EWH6301Y,CAN,GUANGZHOU,23.392436,113.298786,DFW,DALLAS/FORT WORTH,32.896828,-97.037997
5,ULZ8130D,CAN,GUANGZHOU,23.392436,113.298786,DFW,DALLAS/FORT WORTH,32.896828,-97.037997
6,QHU1140O,CDG,PARIS,49.012779,2.550000,LAS,LAS VEGAS,36.080056,-115.152250
7,TMV7633W,CGK,JAKARTA,-6.125567,106.655897,DXB,DUBAI,25.252778,55.364444
8,DAU2617A,CGK,JAKARTA,-6.125567,106.655897,SFO,SAN FRANCISCO,37.618972,-122.374889
9,WSK1289Z,CLT,CHARLOTTE,35.214000,-80.943139,DEN,DENVER,39.861656,-104.673178


In [248]:
# Calculate nautical miles (a^2+b^2=c^2)
from_dest_lat_lon['Miles'] = np.sqrt((from_dest_lat_lon['Dest_Lat']-from_dest_lat_lon['From_Lat'])**2
+(from_dest_lat_lon['Dest_Lon']-from_dest_lat_lon['From_Lon'])**2)

In [ ]:
# #export results
# from_dest_lat_lon.to_csv(r'from_dest_lat_lon.csv')

In [249]:
# Number of miles per flight 
miles_flight = from_dest_lat_lon[['Flight_id','Miles']]
miles_flight

,Flight_id,Miles
0,ATT7791R,110.142627
1,FYL5866L,198.665885
2,XOY7948U,85.841112
3,YZO4444S,181.442562
4,EWH6301Y,210.551409
5,ULZ8130D,210.551409
6,QHU1140O,118.410620
7,TMV7633W,60.128310
8,DAU2617A,233.170936
9,WSK1289Z,24.180890


In [250]:
#Add flight miles to passenger data
obj4_data = pass_data_route.merge(miles_flight, how='outer', on=['Flight_id'])
obj4_data

,Passenger_id,Flight_id,Route,From_airport_IATA,Dest_airport_IATA,Depart_time,Miles
0,UES9151GS5,SQU6245R,DEN-FRA-17:14:20,DEN,FRA,17:14:20,113.671693
1,UES915*GS5,SQU6245R,DEN-FRA-17:14:20,DEN,FRA,17:14:20,113.671693
2,JBE2302VO4,SQU6245R,DEN-FRA-17:14:20,DEN,FRA,17:14:20,113.671693
3,HCA3158QA6,SQU6245R,DEN-FRA-17:14:20,DEN,FRA,17:14:20,113.671693
4,XFG5747ZT9,SQU6245R,DEN-FRA-17:14:20,DEN,FRA,17:14:20,113.671693
...,...,...,...,...,...,...,...
495,HCA3158QA6,XOY7948U,ATL-LHR-17:07:18,ATL,LHR,17:07:18,85.841112
496,WBE6935NU$,XOY7948U,ATL-LHR-17:07:18,ATL,LHR,17:07:18,85.841112
497,SPR4484HA6,XOY7948U,ATL-LHR-17:07:18,ATL,LHR,17:07:18,85.841112
498,VZY2993ME1,XOY7948U,ATL-LHR-17:07:18,ATL,LHR,17:07:18,85.841112


In [253]:
pass_mil_flight = obj4_data[['Passenger_id','Miles']]

In [284]:
pass_mil_flight

,Passenger_id,Miles
0,UES9151GS5,113.671693
1,UES915*GS5,113.671693
2,JBE2302VO4,113.671693
3,HCA3158QA6,113.671693
4,XFG5747ZT9,113.671693
...,...,...
495,HCA3158QA6,85.841112
496,WBE6935NU$,85.841112
497,SPR4484HA6,85.841112
498,VZY2993ME1,85.841112


In [303]:
key2val2 = pd.DataFrame()
for index,row in pass_mil_flight.iterrows():
    key2val2=key2val2.append(pass_mil_flight.loc[index])
    mapper = pd.DataFrame(key2val2, columns=['Miles','Passenger_id'])
#mapper

          Miles Passenger_id
0    113.671693   UES9151GS5
1    113.671693   UES915*GS5
2    113.671693   JBE2302VO4
3    113.671693   HCA3158QA6
4    113.671693   XFG5747ZT9
..          ...          ...
495   85.841112   HCA3158QA6
496   85.841112   WBE6935NU$
497   85.841112   SPR4484HA6
498   85.841112   VZY2993ME1
499   85.841112   YMH6360YP0

[500 rows x 2 columns]


In [316]:
#reducer - takes key and val from mapper to provide final key and value
miles_per_pass=mapper.groupby(['Passenger_id']).sum().round(1)
#reducer
miles_per_pass.reset_index(inplace=True)
# miles_per_pass

In [315]:
# clean up results
#num_passengers=num_flights.fillna(0)
miles_per_pass=miles_per_pass.sort_values(by=['Miles'], ascending=[False])
miles_per_pass.set_index('Passenger_id', inplace=True, drop=True)
miles_per_pass

,Miles
Passenger_id,
UES9151GS5,3378.5
DAZ3029XA0,3305.9
BWI0520BG6,3272.9
PUD8209OG3,3096.7
SPR4484HA6,2906.1
WYU2010YH8,2778.1
JJM4724RF7,2686.0
WBE6935NU3,2514.9
HCA3158QA6,2505.9


# old code, maybe useful

In [ ]:
pass_data_df.value_counts(['From_airport_IATA'])

In [ ]:
pass_data_df.groupby(['From_airport_IATA']).count()

In [ ]:
flight_only_grp.isin(flight_only_grp['From_airport_IATA'])

In [ ]:
#pass_data_df.set_index(["From_airport_IATA"]).count(level="From_airport_IATA")
pass_data_df.count(axis="From_airport_IATA")


In [ ]:
#pd.concat([obj_1_list_df, from_counts_df], keys=['Airport_IATA_code','Airport_IATA_code'])


In [ ]:
#pass_data_df['Depart_time_epoch'].dt.strftime("%m/%d/%y")
#pass_data_df['Depart_time_epoch']=
(pd.to_datetime(pass_data_df['Depart_time_epoch'],unit='s')) 


In [ ]:
obj1 = obj_1_list_df.merge(from_counts_df, how='outer', on=['Airport_IATA_code', 'Airport_IATA_code'])

In [ ]:
if re.match(r"hello[0-9]+", 'hello1'):
    print('Yes')

In [ ]:
pattern = r"[A-Z][A-Z][A-Z]"

pd.Series(
     ["DEN", "A;S", "AM%", "3b", "03c", "4dx"],
 ).str.contains(pattern)

In [ ]:
#pass_data_df[pass_data_df['From_airport_IATA'].str.match('DEN')== True]
#pass_data_df['From_airport_format']=
pass_data_df['From_airport_IATA'].str.match('[A-Z][A-Z][A-Z]')

In [ ]:
#merging by IATA code
#select remaining from airports
from_values = flight_info_true.From_airport_IATA.unique()
dest_values = flight_info_true.Dest_airport_IATA.unique()
#turn to dataframe
from_values_df = pd.DataFrame(from_values)
dest_values_df = pd.DataFrame(dest_values)
#rename columns to allow merge
from_values_df.columns = ['Airport_IATA_code']
dest_values_df.columns = ['Airport_IATA_code']

In [ ]:
from_merge['From_airport_exists']=from_merge['Airport_name'].str.contains(r"[A-Z]")
dest_merge['Dest_airport_exists']=dest_merge['Airport_name'].str.contains(r"[A-Z]")
#from_merge[from_merge['Airport_name'].str.contains('NaN')==False]
#new_df = df[df["col"].str.contains(word) == False]

In [ ]:
import Levenshtein as lev


In [ ]:
import pandas as pd
from textdistance import levenshtein

attempts = [['passw0rd', 'pasw0rd'],
            ['passwrd', 'psword'],
            ['psw0rd', 'passwor'],
            ['helloworld', 'heloworl']]

df=pd.DataFrame(attempts, columns=['password', 'attempt'])

df.apply(lambda x: levenshtein.distance(x['password'],  x['attempt']), axis=1)

In [ ]:
#best_match_results_pd[['results','true']] = 
best_match_results_pd['results'].str.split('-',expand=True)
#df[['V','allele']] = df['V'].str.split('-',expand=True)


In [ ]:
best_match_results_pd.iloc[1, 0]

In [ ]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
similar("Apple","Appel")

In [ ]:
round(SequenceMatcher(None, 'WBE6935NU', 'HCA3158QA6').ratio(),3)
round(SequenceMatcher(None, 'UES9151GS5', 'UES915*GS5').ratio(),3)
round(SequenceMatcher(None, 'UES915*GS5', 'UES9151GS5').ratio(),3)


In [ ]:
install python-Levenshtein

In [ ]:
f = 'SPRb484HA6'
test_best_true = pd.DataFrame()
for t in true:
    ratio = round(SequenceMatcher(None, t, f).ratio(),3)
    print(t, ratio)
    #test_best_true = test_best_true.append(ratio)


In [ ]:
#final_true
#str(final_true)[1:-1] 
#test_best_true
#max_true

In [ ]:
# #Calculate epoch Arrival Time
# pass_data['Arrive_epoch'] = pass_data['Depart_time_epoch']+pass_data['Flight_time_min']*60
# #Convert epoch to datetime
# pass_data['Depart_time_datetime'] = pd.to_datetime(pass_data['Depart_time_epoch'],unit='s')
# pass_data['Arrive_time_datetime'] = pd.to_datetime(pass_data['Arrive_epoch'],unit='s')

In [ ]:
# #And new datetime formats
# pass_data['Depart_day_month']=pass_data['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y")
# #pass_data_df['Depart_day_datetime']=pass_data_df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y %H:%M:%S")
# pass_data['Depart_time']=pass_data['Depart_time_datetime'].dt.strftime("%H:%M:%S")
# pass_data['Arrive_time']=pass_data['Arrive_time_datetime'].dt.strftime("%H:%M:%S")

In [ ]:
# route_info=pass_data[['Route'
#                       ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_day_month'
#                       ,'Depart_time'
#                       ,'Flight_time_min'
#                       ,'Arrive_time'
#                      ]]

# route_info=route_info.groupby(['Route'
#                       ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_day_month'
#                       ,'Depart_time'
#                       ,'Flight_time_min'
#                       ,'Arrive_time'          
#                          ]).count()
# route_info.reset_index(inplace=True)


In [ ]:
# #select subset of data 
# flight_only_data = pass_data[['Flight_id'
#                       ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_day_month'
#                       ,'Depart_time'         
#                       ,'Flight_time_min'
#                       ,'count'
#                         ]]
# 
# #Group data by flight
# flight_only_data = flight_only_data.groupby(['Flight_id'
#                           ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_day_month'
#                       ,'Depart_time'         
#                       ,'Flight_time_min'           
#                          ]).count()
# #reset index
# flight_only_data.reset_index(inplace=True)

# #checks
# flight_only_data
# len(flight_only_data)


In [ ]:
# #select subset of data 
# flight_only_data = merge_with_dest[['Flight_id'
#                       ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_time'
#                       ,'count'
#                         ]]
# #Group data to see total flights
# flight_only_data = flight_only_data.groupby(['Flight_id'
#                         ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_time'
#                          ]).sum()
# flight_only_data

In [ ]:
#send to csv for checking
#merge_with_dest.to_csv(r'merge_with_dest.csv')

In [ ]:
#pass_data_route.to_csv((r'pass_data_route.csv'))
# pass_data_route

In [33]:
#flight_only_data['Route'] = flight_only_data['From_airport_IATA']+'-' + flight_only_data['Dest_airport_IATA']+'-' + flight_only_data['Depart_time']

In [13]:
#Add column to allow counting of passengers
# pass_data['count']='1'

#### For debugging only

In [5]:
# pass_data = pd.read_csv('AComp_Passenger_data_no_error.csv',header=None)   #comment out as necessary
# pass_datet = pd.read_csv('AComp_Passenger_data_no_error_DateTime.csv',header=None)
# pass_datetime.columns = ['Passenger_id'
#                         ,'Flight_id'
#                         ,'From_airport_IATA'
#                         ,'Dest_airport_IATA'
#                         ,'Departure_date_time'
#                         ,'Depart_time_epoch'
#                         ,'Flight_time_min'
#                         ,'Arrive_date_time'
# ]

#### Map Reduce testing

In [ ]:
#map flights to key value pair [airport,1]
key2val2 = []
for a in from_list:
    key2val2.append([a,1])

key2val2
df=pd.DataFrame(key2val2,columns=['key','value'])
df
#reduce
#sum(key2val2)

In [190]:
#map flights to key value pair [airport,1]
val2sum =[]
for a in from_list:
    val2 =1
    val2sum.append(val2)
#reduce
sum(val2sum)

500

#### Obj 1 old method

In [ ]:
# List of top 30 airports
airport_list = airport_lat_lon['Airport_IATA_code']
airport_list.columns = ['From_airport_IATA']
airport_list = pd.DataFrame(airport_list)

In [ ]:
# Add all route info
pass_data_route=pass_data_route.merge(route_info, how='inner', on=['Route'])

In [ ]:
# Count number of departure flights
from_df = pass_data_route['From_airport_IATA']
from_counts = from_df.value_counts()
from_counts_df = pd.DataFrame(from_counts)
# move index to column
from_counts_df['Airport_IATA_code'] =from_counts_df.index

In [ ]:
#Check data
print(from_df)
print(from_counts)
print(from_counts_df)

#### trying to do mapreduce obj 4

In [ ]:
# #mapper, where key = airport and value = 1 for each airport in from_list
# key2val2 = [[a, 1] for a in flights]
# mapper = pd.DataFrame(key2val2, columns=['Flight_id','value'])
# #mapper

# #reducer - takes key and val from mapper to provide final key and value
# num_passengers=mapper.groupby(['Flight_id']).sum()
# #num_passengers
# #reducer
# num_passengers.reset_index(inplace=True)

In [277]:
key2val2 = pd.DataFrame()
key2val2.append(pass_mil_flight, ignore_index=True)

# for index,row in pass_mil_flight.iterrows():
#     key2val2.append(pd.Series(['Passenger_id','Miles'], index=pass_mil_flight.columns ), ignore_index=True)
#     print(row['Passenger_id'],row['Miles'])
#     df=pd.DataFrame(key2val2,columns=['Passenger_id','Miles'])

    
# >>> df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
# >>> df
#    A  B
# 0  1  2
# 1  3  4
# >>> df2 = pd.DataFrame([[5, 6], [7, 8]], columns=list('AB'))
# >>> df.append(df2)
#    A  B
# 0  1  2
# 1  3  4
# 0  5  6
# 1  7  8    
    
    
    
#     modDfObj = dfObj.append(pd.Series(['Raju', 21, 'Bangalore', 'India'], index=dfObj.columns ), ignore_index=True)
#modDfObj = dfObj.append({'Name' : 'Sahil' , 'Age' : 22} , ignore_index=True)

    #map flights to key value pair [airport,1]
# key2val2 = []
# for a in from_list:
#     key2val2.append([a,1])

# key2val2
# df=pd.DataFrame(key2val2,columns=['key','value'])
# df
#reduce
#sum(key2val2)
# key2val2
# df=pd.DataFrame(key2val2,columns=['Passenger_id','Miles'])
# df
#reduce
#sum(key2val2)

# import pandas as pd

# df = pd.DataFrame({'c1': [10, 11, 12], 'c2': [100, 110, 120]})



,Passenger_id,Miles
0,UES9151GS5,113.671693
1,UES915*GS5,113.671693
2,JBE2302VO4,113.671693
3,HCA3158QA6,113.671693
4,XFG5747ZT9,113.671693
...,...,...
495,HCA3158QA6,85.841112
496,WBE6935NU$,85.841112
497,SPR4484HA6,85.841112
498,VZY2993ME1,85.841112


In [ ]:
# import pandas as pd

# df = pd.DataFrame({'c1': [10, 11, 12], 'c2': [100, 110, 120]})

# for index, row in df.iterrows():
#     print(row['c1'], row['c2'])

In [317]:
#mapper, where key = airport and value = 1 for each airport in from_list
# key2val2 = [[p, m] for p in pass_mil_flight]
# mapper = pd.DataFrame(key2val2, columns=['Passenger_id','Miles'])
# #mapper

# #reducer - takes key and val from mapper to provide final key and value
# num_passengers=mapper.groupby(['Flight_id']).sum()
# #num_passengers
# #reducer
# num_passengers.reset_index(inplace=True)

#### Obj 4 old way

In [ ]:
#Number of miles per flight per passenger
pass_flights= pd.pivot_table(obj4_data, values='Miles', index=['Passenger_id'],
               columns=['Flight_id']
              , aggfunc=np.sum
              )
pass_flights

In [ ]:
#pass_flights.to_csv(r'pass_flights.csv')

In [ ]:
#Add up flight miles per passenger
Pass_miles = pass_flights.sum(axis = 1, skipna = True) 
Pass_miles_df = pd.DataFrame(Pass_miles)
Pass_miles_df.columns = ['Air_miles']
Pass_miles_df = Pass_miles_df.sort_values(by=['Air_miles'], ascending=False)
Pass_miles_df.to_csv(r'Pass_miles_df.csv')
Pass_miles_df